# Lyfe - Une certaine définition du vivant

Projet informatique mené par le Grand Vass et le modeste Till dans le cadre de l'UE "Projet numérique" pendant l'année scolaire 2021-2022. Ce rapport est rédigé sur Jupyter Notebook et donc interactif, n'oubliez pas d'executer les cellules dans l'ordre !

# Définition du problème

## Introduction

Nous cherchons dans ce projet à reproduire l'emergence et le comportement de structure bien connus de la nature : les motifs de Turing. Ces motifs nous sont familiers, on les retrouve par exemple sur le poisson globe-géant, la fourrure d'un léopard ou encore sur certains coquillages. L'origine de ces motifs très réguliers n'est pas encore parfaitement comprise car elle repose sur des mécanismes biologique complexe : la morphogénèse, la différenciation, etc. Néanmoins, le mathématician Alan Turing proposa en 1952 dans son essai "The Chemical Basis of Morphogenesis" une première explication. Il suggère une modélisation thermochimique simple à ce phénomène. Sous ce prisme, il démontre que les motifs de Turing émergent naturellement comme solution des équations de réactions-diffusions. Nous allons tenter de résoudre ces équations pour visualiser ces motifs.

<img src="globe.png"  width="1000" />

## Modélisation Théorique Succinte

En réalité, les équations de diffusion-réactions sont une grande classe d'équations aux dérivées partielles et nous n'allons pas chercher à résoudre chacun d'elle. On retrouve parmi elle le système d'équation de Gray-Scott dont nous rappelons la construction :considérons 2 espèces chimique V et U (et une troisième espèce P qui est un déchet de réaction). Les équations chimique qui régissent leurs évolution sont les suivantes :

In [9]:
%%latex
\begin{align}
U + 2V & \rightarrow 3V \\
V & \rightarrow P \\
\end{align} 

<IPython.core.display.Latex object>

De ces équations nous déduisons l'évolution dans le temps et dans l'espace des espèces U et V, ce sont les équations de Gray-Scott.

In [8]:
%%latex
\begin{align}
\frac{\partial U}{\partial t} & = D_{u}\Delta U - UV^{2} + F(1-U)\\
\frac{\partial V}{\partial t} & = D_{v}\Delta V + UV^{2} - (F+k)V\\
\end{align} 

<IPython.core.display.Latex object>

Sans rentrer dans la construction de ces équations, nous pouvons identifier 2 phénomènes couplés dans ces équations. D'une part les phénomènes de diffusion thermique reconnaissable par la présence de l'opérateur Laplacien $\Delta$ et des coefficients de diffusion $D_{u}$ et $D_{v}$. D'autre part des phénomènes de cinétique chimique caractérisés par le coefficients d'annihilation $k$ et de création $F$.

## Objectif du projet

Ces équations, de part leurs structures, n'ont pas de solution analytique. L'emploi d'une résolution numérique semble alors justifiée. Le chercheur John E. Pearson, du laboratoire national de Los Alamos, proposa dans un article qui fait désormais référence, des résolutions de ces équations pour différents paramètres $F$ et $k$. Il dresse notamment un "diagramme de phase" qui repertorie quels motifs régulier émergent en fonction de ces paramètres. Concrètement, notre objectif sera d'obtenir les mêmes résultats. Nous chercherons dans un second temps à analyser tout ces motifs pour essayer de trouver des comportements généraux caractéristiques. En d'autres termes, notre rapport se concentrera dans un premier temps sur l'implémentation de la résolution numérique et dans un second temps de l'analyse physique de nos simulations.

# Exécution du projet

## Outils utilisés

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from PyQt5 import QtWidgets
import time
import datetime
import os
import scipy.optimize as opt

Notre projet se portait très bien à une résolution par approche procédurale. Notre code appelera à tour de rôle les fonctions qui lui permettront de calculer et d'afficher les résultats souhaités. Nous utilisons l'arsenal usuel de librairies python:
- Numpy et scipy.optimize pour les calculs :
- Matplotlib et PyQt5 pour l'affichage et l'animation
- Time, datetime et os pour l'organisation des données récoltées

Explicitions désormais les différentes fonctions que nous avons utilisés.

### La fonction opérateur laplacien $\Delta$

In [5]:
def lap(M, dx, dy):
    """Permet d'obtenir le laplacien discret de la matrice M"""
    L = - (2*M)/(dx**2) - (2*M)/(dy**2)
    L += (np.roll(M, (0,-1), (0,1)))/(dx**2) # voisin de droite
    L += (np.roll(M, (0,+1), (0,1)))/(dx**2) # voisin de gauche
    L += (np.roll(M, (-1,0), (0,1)))/(dy**2) # voisin du dessus
    L += (np.roll(M, (+1,0), (0,1)))/(dy**2) # voisin du dessous
    return L


Le Laplacien est un opérateur différentiel d'ordre 2. Nous avons l'avons implementé à partir de sa définition par la méthode des différences finies. La fonction va dans un premier temps retrancher dans les deux directions par la résolution spatiale. A chaque appel, le laplacien sera calculé en repoussant les bords de la matrice grâce aux fonctions $\texttt{np.roll}$. On évite ainsi les effets de bors.
Nous avions tenté une première implémentation du laplacian en utilisant des combinaisons de $\texttt{np.grad}$ et en reconstruisant l'identité $ \Delta = \overrightarrow{\nabla} \cdot \overrightarrow{\nabla} $
Pour le reste, je suis en train de décortiquer internet, mais je suis pas certain de trouver une réponse convaincante...

### Les équations de Gray-Scott

In [6]:
def G_S(u, v, f, k, Du, Dv, dx, dy):
    """Renvoie les valeurs des derivees temporelles des concentrations en U et␣
    ,→V selon le modele de reaction-diffusion de Gray-Scott"""
    """u et v sont des matrices contenant les concentrations en U et V (en mol.
    ,→L¹), f est le "feed-rate"(en s¹), k (en s¹) est le taux de conversion de U␣
    ,→(en P, inerte), et Du (resp. Dv) est le coefficient de diffusion de U (resp.␣
    ,→V) (en m².s¹)."""
    u_dot = Du * lap(u, dx, dy) - u * v**2 + f * (1-u)
    v_dot = Dv * lap(v, dx, dy) + u * v**2 - (f + k) * v
    return (u_dot, v_dot)

Il s'agit de la fonction centrale de notre programme puisqu'elle implémente les équations de gray-scott présentées en première partie de ce rapport. Il ne s'agit en rien d'autre que les expressions mathématiques de ces dernières. Cette fonction sera ensuite appelée dans un schéma de résolution de type Euler. Nous reviendrons dans un autre temps sur cette méthode de résolution.

### Fonction de régréssion polynomiale

In [6]:
def courbe_fit(t,A,B, C):
    return A*t**0.5 + B*t

Fonction de tracé. Est-ce qu'on mets pas ce paragraphe dans la 3eme section à propos du traitement ? 

### Introduction des variables

In [5]:
d = str(datetime.datetime.today())
date_heure = d[:10]+'__' + d[-15:-13]+'_' + d[-12:-10]+'_'+d[-9:-7]
dt = 1
duree = 10000
Npas = int(duree / dt) # nombre de pas
Nx = 512
Ny = 512
dx = 1e-2
dy = 1e-2
Lx = dx * Nx # espace selon x
Ly = dy * Ny # espace selon y
F = 0.0507
k = 0.0617
Ds = 2e-5
Dg = 1e-5
loc, scale = 0, 0.05
N_disp_save = 1000
N_calc_mean = 1000
save_ok = 1
circle_ok = 1
tracking_ok = 1

Afin de s'inscrire complètement dans la démarche du physicien numéricien, il était important de rendre l'exploitation de notre code la plus autonome possible. Nous devions faire en sorte que notre programme puisse fonctionner sans avoir à ouvrir le code dans un éditeur. Autrement dit, nous devions pouvoir récupérer les données uniquement en éxécutant le code et en insérant un fichier de données externe. Les lignes de codes précédentes remplissent ce role. Dans notre routine, il nous suffisait simplement de modifier ces données, enregistrées en $\texttt{.txt}$ , d'une simulation à l'autre.

### Initialisation

In [10]:
# On initialise le canevas:
xx = np.linspace(0, Lx, Nx)
yy = np.linspace(0, Ly, Ny)
# On initialise deux tableaux portant chacun les valeurs, en tout point, des concentrations en U et en V
conc_S = np.ones((Nx, Ny))
conc_G = np.zeros((Nx, Ny))
# On perturbe cet etat de depart, en changeant la concentration en V d'un carré␣au centre, de cote 2d, 
# que l'on perturbe avec un bruit blanc gaussien
d = 10
conc_S[Nx//2 - d: Nx //2 + d, Ny//2 - d: Ny //2 + d] = 0.50
conc_G[Nx//2 - d: Nx //2 + d, Ny//2 - d: Ny //2 + d] = 0.25 + np.random.normal(loc, scale, size=(2*d, 2*d))
# perturbation gaussienne
#On crée un environement dans lequel les c.i. sont parfaitement symétriques
conc_S_NP = np.ones((Nx, Ny))
conc_G_NP = np.zeros((Nx, Ny))
conc_S_NP[Nx//2 - d: Nx //2 + d, Ny//2 - d: Ny //2 + d] = 0.50
conc_G_NP[Nx//2 - d: Nx //2 + d, Ny//2 - d: Ny //2 + d] = 0.25
#On crée un liste qui stockera les moyennes de la valeur absolue des différences entres les tableaux perturbés et non perturbés
deviation=np.zeros(Npas)
#On calcule le coût en temps des différentes opérations
temps_total = 0
temps_calcul = 0
temps_generation = 0
S = []
G = []
theta = np.linspace(0, np.pi*2, 100)
rayons = []


On génère ici tout les objets qui nous serviront pour nos simulations. Physiquement, $\texttt{cons_S}$ et $\texttt{cons_G}$ correspondent aux espèces chimiques dont on va calculer la répartition dans l'espace au cours du temps. Initialement, on place ces deux espèces au centre de notre aire d'intêret. On choisi ensuite de perturber une des deux espèces par un bruit Gaussien pour créer un déséquilibre. Ce déséquilibre lance la réaction qui sera calculé par les équations de Gray-Scott. 

### Evolution et création des données

In [11]:
insérer le beau code complet ici

SyntaxError: invalid syntax (Temp/ipykernel_76888/1067656890.py, line 1)

Cette partie du code est la plus essentielle. C'est ici que sont calculés, représentés et sauvegardés les évolution physique de nos espèces. On effectue la même suite d'opérations :

- Résolution des équations de Gray-Scott par la méthode d'euler
- Sauvegarde des images tout les $\texttt{N_disp_save}$ pas de temps
- Traitement des données : Evolution du rayon moyen et/ou tracé de la transformée de Fourrier 2D
- Affichage et animation des données et du temps de calcul entre chaque itération

Dernière partie à insérer et commenter sur le rayon.


# Autour de notre programme

## La résolution par la méthode d'Euler

Nous l'avons vu, le choix que nous avons adopté était la méthode d'Euler car c'est un schéma de résolution très facile à implémenter. Pour autant, en programmation, rien n'est gagné d'avance. Il est arrivé à plusieurs reprises que notre programme cesse de fonctionner au bout de quelques itérations. Il nous a fallu sonder davantages les équations que nous tentions de résoudre pour comprendre d'ou venait ce phénomène. Il est important de comprendre que le choix du pas de temps et d'espace est corrélé et qu'il existe un critère de stabilité numérique pour choisir ces grandeurs. Cependant, nos équations étant non linéaire et couplée, déterminer l'expression de ce critère relevait davantages de l'analyse numérique que du coeur de notre projet.
<br>
Notre équation ressemblant fortement à une équation de la chaleur, nous avons trouvé judicieux d'utiliser le même critère de choix de ces pas de temps et d'espace pour notre programme.

$\begin{equation}\Delta t \leq \frac{{\Delta x}^{2}}{2 D_{U,V}}\end{equation}$